<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<br>
<b>Displaying images using Firefly</b> <br>
Last verified to run on 2022-05-31 with LSST Science Pipelines release w_2022_22<br>
Contact authors: Jeff Carlin <br>
Target audience: All DP0 delegates. <br>
Container Size: medium <br>
Questions welcome at <a href="https://community.lsst.org/c/support/dp0">community.lsst.org/c/support/dp0</a> <br>
Find DP0 documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a> <br>

**Credit:** This tutorial is based in part on the `Firefly.ipynb` notebook that is available under your home directory, in `notebooks/notebook-demo/`.



### Learning Objectives

This notebook is a counterpart to tuturial notebook `03_Image_Display_and_Manipulation.ipynb`, and is intended to demonstrate the [Firefly](https://pipelines.lsst.io/v/daily/modules/lsst.display.firefly/index.html) interactive interface for viewing image data. As in the complementary "Image display" notebook, we will use the Butler to access images. We will also use the `lsst.afw.display` library to display images, but in this case with a different "back end" -- the interactive Firefly display tool.

### Set Up

In [ ]:
# What version of the Stack are we using?
! echo $IMAGE_DESCRIPTION
! eups list -s | grep lsst_distrib

### 1. Load the LSST Science Pipelines

First, we load the `lsst.afw.display` library to gain access to the image visualization routines we'd like to use, and the `lsst.daf.butler` library, which is used to access data products.

In [ ]:
# load lsst.afw.display to gain access to image visualization routines.
import lsst.afw.display as afwDisplay
# load the Butler, which provides programmatic access to LSST data products.
from lsst.daf.butler import Butler

And let the kernel know that we're happy not to have some useful warnings printed during this tutorial.

In [ ]:
# prevent some helpful but ancillary warning messages from printing
#   during some LSST DM Release calls
import warnings
warnings.simplefilter("ignore", category=UserWarning)

### 2. Load the Data to Visualize

To display an image, we must first load some data. These data have been processed with the LSST Science Pipelines, and are organized in a structure that enables us to access them through the `Butler`. For more information on the `Butler`, see [lsst.daf.butler](https://pipelines.lsst.io/modules/lsst.daf.butler/index.html).

The DP0.1 data set contains simulated images from the LSST DESC Data Challenge 2 (DC2). These data are available in an S3 bucket `s3://butler-us-central1-dp01`. We access a single image from a specific visit (`192350`) and detector (`175`).

Once we define a string that contains the data directory, we start the `Butler` instance using the `lsst.daf.butler` library and its `Butler` class. The `Butler` object is initialized with a string containing the data directory we wish to access. Running the cell may take a few moments.

With the `Butler` instance now generated using our data directory, we can retrieve the desired calibrated exposure by telling the butler which filter ("band"), CCD ("detector"), and visit we wish to view. To do this, we define a dictionary with the required information.

In [ ]:
dataId = {'visit': 192350, 'detector': 175, 'band': 'i'}
# Note: because the combination of visit+detector already uniquely identifies
# the exposure, specifying "band" above is unnecessary.

repo = 's3://butler-us-central1-dp01'
collection = '2.2i/runs/DP0.1'
butler = Butler(repo, collections=collection)

# Retrieve the data using the `butler` instance and its function `get()`
calexp = butler.get('calexp', **dataId)

# Note: This will trigger a warning from CFITSIO in w_2022_22. This warning can be safely ignored and will be corrected in the future.

## Create a Display

To display the `calexp` you will use the LSST `afwDisplay` framework. It provides a uniform API for multiple display backends, including DS9, matplotlib, and LSST’s Firefly viewer. The default backend is `ds9`, but since we are working remotely on `JupyterLab` we would prefer to use the web-based Firefly display. A [user guide](https://pipelines.lsst.io/v/daily/modules/lsst.display.firefly/index.html)  for `lsst.display.firefly` is available on the [pipelines.lsst.io](https://pipelines.lsst.io/v/daily) site.

Now, we create a Firefly display.

In [ ]:
afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.Display(frame=1)
afw_display.getClient().show_lab_tab()

In the Science Platform Notebook aspect, a Firefly viewer tab appears. You may wish to drag it to the right side of the JupyterLab area to create two side by side panes, one with the notebook and one with the display.

## Display the calexp (calibrated exposure)

We can now build the display and use the `mtv` method to view the `calexp` with Firefly. First we display an image with mask planes and then overplot some sources.

In [ ]:
afw_display.mtv(calexp)

As soon as you execute the command a single calibrated HSC exposure for the `{'band': 'i', 'detector': 175, 'visit': 192350}` data ID should appear in the Firefly `JupyterLab` tab.

Notice that the image is overlaid with colorful regions. These are mask regions. Each color reflects a different mask bit that correspond to detections and different types of detector artifacts. You’ll learn how to interpret these colors later, but first you’ll likely want to adjust the image display.

## Improving the image display

The display framework gives you control over the image display to help bring out image details. 
For example, to make masked regions semi-transparent, so that underlying image features are visible, try:

In [ ]:
afw_display.setMaskTransparency(80)

The setMaskTransparency method’s argument can range from 0 (fully opaque) to 100 (fully transparent).

You can also control the colorbar scaling algorithm with the display’s scale method. Try an asinh stretch with explicit minimum (black) and maximum (white) values:

In [ ]:
afw_display.scale("asinh", -5, 20)

You can also use an automatic algorithm like `zscale` (or `minmax`) to select the white and black thresholds:

In [ ]:
afw_display.scale("asinh", "zscale")

## Interpreting displayed mask colors

The display framework renders each plane of the mask in a different color (plane being a different bit in the mask). To interpret these colors you can get a dictionary of mask planes from the `calexp` and query the display for the colors it rendered each mask plane with. For example:

In [ ]:
mask = calexp.getMask()
for maskName, maskBit in mask.getMaskPlaneDict().items():
    print('{}: {}'.format(maskName, afw_display.getMaskPlaneColor(maskName)))

In the Firefly viewer tab, the <a href="https://irsa.ipac.caltech.edu/onlinehelp/finderchart/finderchart/img/layers.png">overlays button</a> on the toolbar gives you detailed control over the mask planes, such as turning individual planes on and off, changing the color and adjusting the transparency. Mask transparency and colors can also be set using `afw.display` commands, for individual planes or for all.

## Plotting sources on the display

The measurements for all sources detected in the `calexp` image by the LSST science pipelines are in a dataset type `src`. Let's use the Butler to get the `src` catalog corresponding to the same `dataId` as the image we displayed:

In [ ]:
src = butler.get('src', **dataId)

The returned object, `src`, is a `lsst.afw.table.SourceTable` object. `SourceTables` are explored more elsewhere, but you can do some simple investigations using common python functions. For example, to check the length of the object:




In [ ]:
len(src)

You can view an HTML rendering of the `src` table by getting an `astropy.table.Table` version of it:

In [ ]:
src.asAstropy()

Now we’ll overplot sources from the `src` table onto the image display using the Display’s `dot` method for plotting markers. `Display.dot` plots markers individually, so you’ll need to iterate over rows in the `SourceTable`.  It is more efficient to send a batch of updates to the display, so we enclose the loop in a `display.Buffering` context, like this:

In [ ]:
with afw_display.Buffering():
    for record in src:
        afw_display.dot('o', record.getX(), record.getY(), size=20, ctype='orange')

## Clearing markers

`Display.dot` always adds new markers to the display. To clear the display of all markers, use the erase method:

In [ ]:
afw_display.erase()

## Display a coadd image

"Coadds" are the images made by combining multiple overlapping `calexp`s to create a deeper image. Let's look at the one of them (in fact, the same coadd that was examined in the related "Image Display" notebook). The Butler dataset type is called a `deepCoadd_calexp`.

In [ ]:
dataId_coadd = {'tract': 4431, 'patch': 17, 'band': 'i'}

# Retrieve the data using the `butler` instance and its function `get()`
coadd_calexp = butler.get('deepCoadd_calexp', **dataId_coadd)

We will also grab the measurements for sources in this coadd (called `deepCoadd_forced_src` because it is "forced" photometry at the positions of all detected objects), and the "reference table" that contains merged information about all detected objects from all filters.

In [ ]:
forced_src = butler.get('deepCoadd_forced_src', **dataId_coadd)
refTable = butler.get('deepCoadd_ref', **dataId_coadd)

In [ ]:
# Display the coadd_calexp image in Firefly:
afw_display.mtv(coadd_calexp)

## Plotting sources on the display

As we did for the `calexp`, let's display sources on top of the `deepCoadd_calexp` image. We will apply three selection criteria:

1. Spatially select objects that are near the bottom left corner of the image (at X<1000 and Y<1000, where X and Y are the pixel values. (This is just so that the overplotting will go faster.)

2. Use the `detect_isPrimary` flag to select non-duplicate objects that have been deblended (see [this section of the pipelines tutorial](https://pipelines.lsst.io/getting-started/multiband-analysis.html#the-go-to-flag-detect-isprimary) for details).

3. Use one of the flags from the reference table that tells you whether a source was measured to be "extended" (like a galaxy). The `base_ClassificationExtendedness_value` is set to 1 for extended sources (galaxies) and 0 for point sources (stars). Again, see the tutorial section linked above for more about this flag and an example of its usage.

Note that if you want to see all of the columns in the `forced_src` or `refTable` catalogs, you can use the method "forced_src.schema" or "refTable.schema".

In [ ]:
# Note: because this "patch" is part of a larger "tract,"
#   the (X, Y) coordinates of the lower-left corner are not (0, 0).
#   We need to get the coordinates of that corner and subtract them off.
xy0 = coadd_calexp.getXY0()
xPos = refTable["base_SdssCentroid_x"] - xy0.getX()
yPos = refTable["base_SdssCentroid_y"] - xy0.getY()

boxSelect = (xPos < 1000) & (yPos < 1000)

isPrimary = refTable['detect_isPrimary']

isStellar = refTable['base_ClassificationExtendedness_value'] < 1.

In [ ]:
# Display stars with orange circles:

with afw_display.Buffering():
    for record in forced_src[boxSelect & isPrimary & isStellar]:
        afw_display.dot('o', record.getX(), record.getY(), size=20, ctype='orange')

In [ ]:
# Display galaxies (really, just anything that is "extended") with blue circles:

with afw_display.Buffering():
    for record in forced_src[boxSelect & isPrimary & ~isStellar]:
        afw_display.dot('o', record.getX(), record.getY(), size=20, ctype='blue')

This tutorial was a basic introduction to displaying and manipulating images with Firefly. Enjoy exploring the DP0 images and catalogs!

### Additional Documentation

If you'd like some more information on `lsst.afw.display`, please have a look at the following websites:

* [Firefly user guide](https://pipelines.lsst.io/v/daily/modules/lsst.display.firefly/index.html)
* [Getting Started on Image Display (pipelines.lsst.io)](https://pipelines.lsst.io/getting-started/display.html)

* [afw.display Doxygen website](http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/namespacelsst_1_1afw_1_1display.html)  
* [afw.display GitHub website](https://github.com/lsst/afw/tree/master/python/lsst/afw/display)  
